In [1]:
import pandas as pd

import numpy as np

import pingouin as pg

import geopandas as gpd

import matplotlib.pyplot as plt
import seaborn as sns
import math

from IPython.display import Markdown as md

In [2]:
from nycschools import schools, ui, exams, geo

In [3]:
df = schools.load_school_demographics()
# schools.search(df, "hack")
df["maker"] = df.school_name.apply(lambda x: "make" in x.lower() or "software" in x.lower())
df[df.maker == True][["dbn","school_name"]].drop_duplicates()

,dbn,school_name
370,02M282,Urban Assembly Maker Academy
739,02M546,Academy for Software Engineering
944,03M191,The Riverside School for Makers and Artists
2744,10X264,Bronx Academy for Software Engineering (BASE)


In [34]:
# load the demographic data and merge it with the math data
nyc_schools = schools.load_school_demographics(refresh=False)
nyc_schools[schools.demo.core_cols].query("geo_district == 0")  
nyc_schools[["zip"] + schools.demo.core_cols]


,zip,dbn,beds,district,geo_district,boro,school_name,ay,total_enrollment,female_n,...,hispanic_pct,white_n,white_pct,swd_n,swd_pct,ell_n,ell_pct,poverty_n,poverty_pct,eni_pct
0,10009,01M015,310100010015,1,1,Manhattan,P.S. 015 Roberto Clemente,2016,178,83,...,0.590000,4,0.022000,51,0.287000,12,0.067,152,0.854,0.882
1,10009,01M015,310100010015,1,1,Manhattan,P.S. 015 Roberto Clemente,2017,190,99,...,0.579000,6,0.032000,49,0.258000,8,0.042,161,0.847,0.890
2,10009,01M015,310100010015,1,1,Manhattan,P.S. 015 Roberto Clemente,2018,174,85,...,0.546000,6,0.034000,39,0.224000,8,0.046,147,0.845,0.888
3,10009,01M015,310100010015,1,1,Manhattan,P.S. 015 Roberto Clemente,2019,190,94,...,0.505000,9,0.047000,46,0.242000,17,0.089,155,0.816,0.867
4,10009,01M015,310100010015,1,1,Manhattan,P.S. 015 Roberto Clemente,2020,193,101,...,0.528000,11,0.057000,43,0.223000,21,0.109,158,0.819,0.856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,10474,84X730,320800860846,84,8,Bronx,Bronx Charter School for the Arts,2016,320,168,...,0.737500,3,0.009375,67,0.209375,51,0.159,235,0.734,0.840
9997,10474,84X730,320800860846,84,8,Bronx,Bronx Charter School for the Arts,2017,314,173,...,0.773885,1,0.003185,68,0.216561,57,0.182,258,0.822,0.891
9998,10474,84X730,320800860846,84,8,Bronx,Bronx Charter School for the Arts,2018,430,232,...,0.746512,3,0.006977,103,0.239535,71,0.165,363,0.844,0.888
9999,10474,84X730,320800860846,84,8,Bronx,Bronx Charter School for the Arts,2019,523,280,...,0.722753,5,0.009560,117,0.223709,69,0.132,453,0.866,0.892


In [ ]:
nys_exams = exams.load_math_ela_long()
nys_exams = nys_exams[nys_exams["mean_scale_score"].notnull()]

regents = exams.load_regents()

regents = df.merge(regents, on=["dbn", "ay"], how="inner")
cols = [x for x in regents.columns if not x.startswith("grade")]
regents = regents[cols]
regents

In [3]:
locs = geo.load_school_locations()
locs.plot()

,dbn,zip,geo_district,district,x,y,administrative_district_code,administrative_district_name,beds,borough_block_lot,...,nta_name,open_date,police_precinct,primary_building_code,principal_name,principal_phone_number,principal_title,state_code,status_descriptions,geometry
0,02M212,10036,2,2,987390,216649,2.0,COMMUNITY SCHOOL DISTRICT 02,310200010212,1.010380e+09,...,Clinton,1999-07-01T00:00:00,18.0,M017,KATHLEEN LOUA,212-247-0208,Principal,NY,Open,POINT (-73.98867 40.76133)
1,29Q034,11429,29,29,1056968,198417,29.0,COMMUNITY SCHOOL DISTRICT 29,342900010034,4.111510e+09,...,Queens Village,1898-07-01T00:00:00,105.0,Q034,PAULINE SHAKESPEARE,718-465-6818,Principal,NY,Open,POINT (-73.73770 40.71099)
2,17K091,11203,17,17,1000700,180396,17.0,COMMUNITY SCHOOL DISTRICT 17,331700010091,3.047940e+09,...,Prospect Lefferts Gardens-Wingate,1904-07-01T00:00:00,71.0,K091,Tessa Alleyne,718-756-0243,Principal,NY,Open,POINT (-73.94071 40.66181)
3,31R455,10312,31,31,930477,131756,31.0,COMMUNITY SCHOOL DISTRICT 31,353100011455,5.066130e+09,...,Annadale-Huguenot-Prince's Bay-Eltingville,1972-07-01T00:00:00,123.0,R455,Joseph Scarmato,718-668-8800,Principal,NY,Open,POINT (-74.19343 40.52815)
4,07X522,10451,7,7,1006407,237542,7.0,None,320700011522,2.024110e+09,...,Melrose South-Mott Haven North,2011-07-01T00:00:00,40.0,X600,JOYCE PULPHUS,718-402-7690,Principal,NY,Open,POINT (-73.91995 40.81865)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099,15K124,11215,15,15,986852,182452,15.0,COMMUNITY SCHOOL DISTRICT 15,331500010124,3.010340e+09,...,Park Slope-Gowanus,1900-07-01T00:00:00,78.0,K124,Maria Interlandi,718-788-0246,I.A. Principal,NY,Open,POINT (-73.99062 40.66746)
2100,28Q161,11419,28,28,1033868,190873,28.0,COMMUNITY SCHOOL DISTRICT 28,342800010161,4.094910e+09,...,Richmond Hill,2000-07-01T00:00:00,102.0,Q161,JILL HODER,718-441-5493,Principal,NY,Open,POINT (-73.82108 40.69044)
2101,08X100,10473,8,8,1022234,238985,8.0,COMMUNITY SCHOOL DISTRICT 08,320800010100,2.036410e+09,...,Soundview-Castle Hill-Clason Point-Harding Park,1961-07-01T00:00:00,43.0,X100,ARIANA ARBOLINO,718-842-1461,Principal,NY,Open,POINT (-73.86276 40.82255)
2102,84M523,10024,3,84,991316,225688,84.0,OFFICE OF CHARTER SCHOOLS,310300861008,1.012150e+09,...,Upper West Side,2011-07-01T00:00:00,20.0,M470,Jennifer Haynes,646-274-1580,Principal,NY,Open,POINT (-73.97448 40.78613)


In [3]:
exams.load_nysed_nyc_exams()

,dbn,ay,exam,grade,category,test_year,mean_scale_score,number_tested,number_not_tested,level_1_n,level_1_pct,level_2_n,level_2_pct,level_3_n,level_3_pct,level_4_n,level_4_pct,level_3_4_n,level_3_4_pct,beds
0,01M015,2020,ela,3,All Students,2021,613.0,8,21,0.0,NaN,2.0,NaN,4.0,NaN,2.0,NaN,42.0,NaN,310100010015
1,01M015,2020,ela,3,All Students,2021,613.0,8,21,0.0,NaN,2.0,NaN,4.0,NaN,2.0,NaN,42.0,NaN,310100010015
2,01M015,2020,ela,3,Female,2021,NaN,5,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310100010015
3,01M015,2020,ela,3,Female,2021,NaN,5,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310100010015
4,01M015,2020,ela,3,Male,2021,NaN,3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,310100010015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345075,84R076,2020,math,3,Not Homeless,2021,NaN,29,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,353100861144
345076,84R076,2020,math,3,Not Migrant,2021,579.0,30,42,18.0,NaN,9.0,NaN,3.0,NaN,0.0,NaN,30.0,NaN,353100861144
345077,84R076,2020,math,3,Not Migrant,2021,579.0,30,42,18.0,NaN,9.0,NaN,3.0,NaN,0.0,NaN,30.0,NaN,353100861144
345078,84R076,2020,math,3,Parent Not in Armed Forces,2021,579.0,30,42,18.0,NaN,9.0,NaN,3.0,NaN,0.0,NaN,30.0,NaN,353100861144
